In [1]:
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np
import time
import ipywidgets.widgets as widgets
from cvlib.object_detection import YOLO
import threading
from identify_target import identify_GetTarget

In [2]:
import Arm_Lib
arm = Arm_Lib.Arm_Device()
joints_0 = [90, 135, 0, 0, 90, 30]
arm.Arm_serial_servo_write6_array(joints_0, 1000)

In [3]:
target = identify_GetTarget()

In [38]:
def scaling(x, y):
        scale_x = 1
        scale_y = 1
        if y<240:
            scale_y = y/240
        if x<320:
            scale_x = x/320
        elif x>320:
            scale_x = (x-320)/3200 + 1

        return scale_x, scale_y

def camera(capture):
    start_time = time.time()
    frame_count = 0
    fps_total = 0
    
    global camera_stop, cntr, obj
    while capture.isOpened() and not camera_stop:
        _, img = capture.read()
        
        if not grab:
            bbox, label, conf = yolo.detect_objects(img)
            yolo.draw_bbox(img, bbox, label, conf, write_conf=True)
            if obj in label:
                index = label.index(obj)
                centre = (int((bbox[index][0] + bbox[index][2])/2), int((bbox[index][1] + bbox[index][3])/2))
                scale_x, scale_y = scaling(centre[0], centre[1])
                cntr = (round((((centre[0]*scale_x) - 320) / 4000), 5), round(((480 - (centre[1]*scale_y)) / 3000) * 0.8+0.19, 5))
            else:
                cntr = None
        cur_time = time.time()
        elapsed = cur_time - start_time
        start_time = cur_time
        fps = 1/elapsed
        frame_count += 1
        fps_total += fps
        avg_fps = fps_total/frame_count
        fps_txt = "FPS: {:.2f}, Avg FPS: {:.2f}".format(fps, avg_fps)
        cv.putText(img, fps_txt, (30, 430), cv.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,255), 1)
        
        image_widget.value = cv.imencode('.jpg', img)[1].tobytes()

    capture.release()
    cv.destroyAllWindows()

In [39]:
def claw():
    global grab
    grab = not grab
    threading.Thread(target=target.target_run, args=(cntr,)).start()
    if not target.move_status:
        grab = not grab

In [41]:
yolo = YOLO('yolov4-tiny-custom_best.weights', 'yolov4-tiny-test.cfg', 'obj.names')
capture = cv.VideoCapture(0)
cntr = None
camera_stop = 0
grab = 0
image_widget = widgets.Image(format='jpeg', width=640, height=480)
display(image_widget)
threading.Thread(target=camera, args=(capture,)).start()

[INFO] Initializing YOLO ..


Image(value=b'', format='jpeg', height='480', width='640')

In [49]:
obj = "yellow"

In [50]:
claw()

In [51]:
claw()

In [52]:
camera_stop = 1